In [ ]:
 
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
import tensorflow as tf

K.clear_session()

In [ ]:
data_entrenamiento = '/content/drive/My Drive/ProyectoVision/data/ENTRENAMIENTO'
data_validacion = '/content/drive/My Drive/ProyectoVision/data/PRUEBA'

In [ ]:
"""
Parameters
"""
epocas=30
longitud, altura = 256, 256
batch_size = 32
pasos = 1000
validation_steps = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 5
lr = 0.0004

In [ ]:
##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 1288 images belonging to 5 classes.
Found 321 images belonging to 5 classes.


In [ ]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

#cnn.add(Dropout(0.5))
#cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
#cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(lr=lr),
            metrics=['accuracy'])

In [ ]:
pasos_entrenamiento = entrenamiento_generador.n//entrenamiento_generador.batch_size
pasos_validacion = validacion_generador.n//validacion_generador.batch_size

In [ ]:
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos_entrenamiento,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=pasos_validacion)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
40/40 [==============================] - 122s 3s/step - loss: 4.8041 - accuracy: 0.3432 - val_loss: 1.5693 - val_accuracy: 0.3375
Epoch 2/30
40/40 [==============================] - 118s 3s/step - loss: 1.3344 - accuracy: 0.4658 - val_loss: 1.6689 - val_accuracy: 0.3063
Epoch 3/30
40/40 [==============================] - 118s 3s/step - loss: 1.2576 - accuracy: 0.4833 - val_loss: 1.6397 - val_accuracy: 0.3406
Epoch 4/30
40/40 [==============================] - 118s 3s/step - loss: 1.1710 - accuracy: 0.5350 - val_loss: 1.8064 - val_accuracy: 0.3875
Epoch 5/30
40/40 [==============================] - 119s 3s/step - loss: 1.0720 - accuracy: 0.5764 - val_loss: 2.1398 - val_accuracy: 0.2937
Epoch 6/30
40/40 [==============================] - 122s 3s/step - loss: 1.0245 - accuracy: 0.5844 - val_loss: 1.9105 - val_accuracy: 0.3906
Epoch 7/30
40/40 [==============================] - 118s 3s/step - loss: 1.001

In [ ]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

In [ ]:
from tensorflow.python.keras.applications.mobilenet import preprocess_input
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(directory='/content/drive/My Drive/ProyectoVision/data/PRUEBA', target_size=(altura, longitud), color_mode='rgb', class_mode=None, batch_size=1, shuffle=False, seed=42)

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=cnn.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
import numpy as np

predicted_class_indices=np.argmax(pred,axis=1)
print(predicted_class_indices)
print(type(predicted_class_indices))

In [ ]:
import pandas as pd
labels = (entrenamiento_generador.class_indices)
labels= dict((v,k) for k, v in labels.items())
predictions = [labels[k]for k in predicted_class_indices] 
filenames=test_generator.filenames
results = pd.DataFrame({"filename":filenames, "Predictions":predictions})
results.to_csv("results1.csv", index=False)

print(entrenamiento_generador.class_indices)